# With this notebook, we will be trying to gain a better understanding of the Purple Air API.

In [3]:
#load in libraries
import requests
import json
import pandas as pd
import io
import datetime
from json import JSONDecodeError





### Testing Api connection

In [4]:
#authentication keys
Read_Key = "C7AF49F7-C3F6-11ED-B6F4-42010A800007"
Write_Key = "C7B07520-C3F6-11ED-B6F4-42010A800007"

In [5]:
# Establish initial connection 

# Set up the API endpoint URL and headers
url = "https://api.purpleair.com/v1/keys"
headers = {"X-API-Key": Read_Key}

# Make the API request
response = requests.get(url, headers=headers)

# Print the API response
print(response.json())


{'api_version': 'V1.0.11-0.0.44', 'time_stamp': 1683903198, 'api_key_type': 'READ'}


#### Connection was successful 

### Next, Test pulling sensor data

How to make a call : https://community.purpleair.com/t/making-api-calls-with-the-purpleair-api/180

How to find sensor index : https://community.purpleair.com/t/sensor-index/4000

In [20]:
#Sensor ID for testing
Union_Bethel_AME = "133730#11.74"
William_S_Schmidt = "134488#11.74"

sensor_id_dict = {"Union_Bethel_AME": "133730#11.74", 
                "William_S_Schmidt": "134488#11.74",
                "PGCPS_Schmidt_CenterBldg": "102898#11",
                "Surratsville_HS": "102852#11.05",
                "Oxon_Hill_HS": "104790#11.05"} # Replace with your own sensor IDs and locations
sensor = sensor_id_dict["Union_Bethel_AME"]

In [7]:
#new url for sensor data
endpoint = "https://api.purpleair.com/v1/sensors/"

# Define API headers with your X-API-Key
headers = {
    "X-API-Key": Read_Key
}

# Make API GET request
response = requests.get(endpoint + Union_Bethel_AME, headers=headers)

# Check if request was successful
if response.status_code == 200:
    # Get data from response
    data = response.json()
    # Print data
    print(data)
else:
    # Print error message
    print("Error:", response.status_code)
    
response.text

{'api_version': 'V1.0.11-0.0.44', 'time_stamp': 1683903198, 'data_time_stamp': 1683903149, 'sensor': {'sensor_index': 133730, 'last_modified': 1676581473, 'date_created': 1634686448, 'last_seen': 1683903080, 'private': 0, 'is_owner': 0, 'name': 'Union Bethel AME', 'icon': 0, 'location_type': 0, 'model': 'PA-II-SD', 'hardware': '2.0+OPENLOG+NO-DISK+DS3231+BME280+PMSX003-B+PMSX003-A', 'led_brightness': 50, 'firmware_version': '7.02', 'rssi': -70, 'uptime': 49242, 'pa_latency': 212, 'memory': 15896, 'position_rating': 5, 'latitude': 38.70654, 'longitude': -76.8886, 'altitude': 236, 'channel_state': 3, 'channel_flags': 0, 'channel_flags_manual': 0, 'channel_flags_auto': 0, 'confidence': 100, 'confidence_auto': 100, 'confidence_manual': 100, 'humidity': 33, 'humidity_a': 33, 'temperature': 92, 'temperature_a': 92, 'pressure': 1012.58, 'pressure_a': 1012.58, 'analog_input': 0.0, 'pm1.0': 9.1, 'pm1.0_a': 8.5, 'pm1.0_b': 9.6, 'pm2.5': 18.0, 'pm2.5_a': 18.4, 'pm2.5_b': 17.7, 'pm2.5_alt': 7.7, '

'{\n  "api_version" : "V1.0.11-0.0.44",\n  "time_stamp" : 1683903198,\n  "data_time_stamp" : 1683903149,\n  "sensor" : {\n    "sensor_index" : 133730,\n    "last_modified" : 1676581473,\n    "date_created" : 1634686448,\n    "last_seen" : 1683903080,\n    "private" : 0,\n    "is_owner" : 0,\n    "name" : "Union Bethel AME",\n    "icon" : 0,\n    "location_type" : 0,\n    "model" : "PA-II-SD",\n    "hardware" : "2.0+OPENLOG+NO-DISK+DS3231+BME280+PMSX003-B+PMSX003-A",\n    "led_brightness" : 50,\n    "firmware_version" : "7.02",\n    "rssi" : -70,\n    "uptime" : 49242,\n    "pa_latency" : 212,\n    "memory" : 15896,\n    "position_rating" : 5,\n    "latitude" : 38.70654,\n    "longitude" : -76.8886,\n    "altitude" : 236,\n    "channel_state" : 3,\n    "channel_flags" : 0,\n    "channel_flags_manual" : 0,\n    "channel_flags_auto" : 0,\n    "confidence" : 100,\n    "confidence_auto" : 100,\n    "confidence_manual" : 100,\n    "humidity" : 33,\n    "humidity_a" : 33,\n    "temperature" :

#### Above is a raw output of the current metrics a sensor has. This seems problematic becuase no one on our team has knowledge on Air quality terms.

#### We will learn these terms to then pull insihgts or at the very least be able to create a table for the data that we can pull. 

#### Next thing to do is attempt to pull multiple sensors at once, 
#### and pull historical data. https://api.purpleair.com/#api-sensors-get-sensor-history

In [8]:
def get_sensor_history(sensor_id):
    url = f"https://api.purpleair.com/v1/sensors/{sensor_id}/history/csv?raw=true&fields=pm2.5_alt&averaging=30"
    response = requests.get(url, headers=headers)
    data = response.json()
    return data

In [9]:
test = get_sensor_history(sensor_id_dict["Union_Bethel_AME"])
type(test)

dict

In [10]:
test_df = pd.DataFrame.from_dict(test)
test_df["sensor"]["stats"]

{'pm2.5': 18.0,
 'pm2.5_10minute': 17.4,
 'pm2.5_30minute': 17.2,
 'pm2.5_60minute': 16.0,
 'pm2.5_6hour': 9.6,
 'pm2.5_24hour': 5.8,
 'pm2.5_1week': 4.2,
 'time_stamp': 1683903080}

In [11]:
def get_purple_history_to_json(sensor, start_date, end_date):
    root_url = f'https://api.purpleair.com/v1/sensors/{sensor}/history'

    my_header = {'X-API-Key': Read_Key, 'Content-Type': 'application/json'}
    my_params = {'start_timestamp': start_date, 'end_timestamp': end_date,
                'fields': 'humidity, temperature, pm2.5_atm, pm2.5_alt'}
    response = requests.get(
        root_url,
        headers=my_header,
        params=my_params,
    )
    try:
        response_json = response.json()
    except JSONDecodeError:
        print('Response could not be serialized')
        return None
    
    return response_json


In [13]:
import datetime

date_string = "05/11/2023"
dt = datetime.datetime.strptime(date_string, "%m/%d/%Y")
timestamp = dt.timestamp()

print(timestamp)

1683777600.0


In [14]:

response_json = get_purple_history_to_json(sensor_id_dict["Union_Bethel_AME"], timestamp, timestamp)
print(response_json)

{'api_version': 'V1.0.11-0.0.44', 'time_stamp': 1683903326, 'data_time_stamp': 1683903269, 'sensor': {'sensor_index': 133730, 'humidity': 33, 'temperature': 91, 'pm2.5_alt': 7.2, 'pm2.5_atm': 16.4}}


In [19]:
response_json["sensor"]["pm2.5_atm"]

16.4

# other

In [27]:
root_url = 'https://api.purpleair.com/v1/sensors'
my_header = {'X-API-Key': Read_Key, 'Content-Type': 'application/json'}
my_params = {'fields': 'pm2.5_cf_1'}

response = requests.get(
    root_url,
    headers=my_header,
    params=my_params,
)
try:
    response_json = response.json()
except json.JSONDecodeError:
    print('Response could not be serialized')
    response_json = None

data = json.dumps(response_json, indent=4)

In [43]:
response_str = data  # insert your response string here
response_json = json.loads(response_str)

sensor_index_list = []
pm25_list = []

for data in response_json['data']:
    sensor_index_list.append(data[0])
    pm25_list.append(data[1])

print(sensor_index_list)
print(pm25_list)

[131075, 131079, 131077, 131083, 131087, 131085, 131091, 131089, 131097, 53, 77, 81, 131190, 131193, 131203, 131201, 131205, 131211, 131255, 182, 131259, 195, 131265, 131279, 131281, 131285, 131295, 131293, 131301, 131305, 131309, 131317, 131323, 131327, 131325, 131335, 131343, 131347, 131355, 131353, 286, 131365, 131379, 314, 322, 324, 328, 131403, 334, 340, 344, 348, 131421, 131429, 131435, 131447, 131449, 131457, 131461, 131479, 131485, 417, 131489, 443, 453, 131525, 131533, 131537, 469, 473, 131547, 131553, 131559, 489, 131563, 131565, 131583, 131587, 131585, 131589, 525, 131611, 131613, 131619, 547, 131627, 131625, 131643, 131647, 131649, 583, 591, 131661, 131665, 131687, 131685, 131693, 131701, 131707, 131709, 131717, 131723, 661, 131745, 131751, 131749, 131753, 131757, 131763, 131787, 131785, 131791, 729, 741, 131815, 131819, 747, 131817, 749, 131825, 757, 131835, 765, 767, 131837, 131855, 131853, 820, 826, 828, 131903, 131901, 131905, 131911, 131909, 131915, 131917, 131923, 131